<a href="https://colab.research.google.com/github/Daniel-Benson-Poe/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/db_LS_DS_432_Convolution_Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/Daniel-Benson-Poe/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/train/mountain/art1131.jpg?raw=1)|![](https://github.com/Daniel-Benson-Poe/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/validation/forest/cdmc317.jpg?raw=1)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [2]:
import tensorflow as tf
import os

_URL = 'https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data.zip?raw=true'

path_to_zip = tf.keras.utils.get_file('./data.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

42172416/42170838 [==============================] - 0s 0us/step


In [3]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [4]:
train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training cat pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training dog pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation cat pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation dog pictures

In [5]:
num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [6]:
print('total training mountain images:', num_mountain_tr)
print('total training forest images:', num_forest_tr)

print('total validation mountain images:', num_mountain_val)
print('total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training mountain images: 254
total training forest images: 270
total validation mountain images: 125
total validation forest images: 62
--
Total training images: 524
Total validation images: 187


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [7]:
batch_size = 16
epochs = 100
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [9]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [10]:
y_train_labels = train_data_gen.labels
y_train_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [11]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


In [12]:
y_val_labels = val_data_gen.labels
y_val_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)

## Instatiate Model

In [42]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, Sequential # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

for layer in resnet.layers:
    layer.trainable = False

x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard

import os

def scheduler(epoch):
  if epoch < 10:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (10 - epoch))

lrs = LearningRateScheduler(scheduler)
logdir = os.path.join("/logs","adadelta" )

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Fit Model

In [13]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[lrs]
)

Epoch 1/50
32/32 [==============================] - 4s 114ms/step - loss: 0.8409 - accuracy: 0.5110 - val_loss: 0.9779 - val_accuracy: 0.3693 - lr: 0.0010
Epoch 2/50
32/32 [==============================] - 2s 78ms/step - loss: 0.7991 - accuracy: 0.5190 - val_loss: 0.9688 - val_accuracy: 0.3295 - lr: 0.0010
Epoch 3/50
32/32 [==============================] - 2s 77ms/step - loss: 0.7778 - accuracy: 0.5137 - val_loss: 0.9109 - val_accuracy: 0.3409 - lr: 0.0010
Epoch 4/50
32/32 [==============================] - 2s 77ms/step - loss: 0.7530 - accuracy: 0.5170 - val_loss: 0.8719 - val_accuracy: 0.3409 - lr: 0.0010
Epoch 5/50
32/32 [==============================] - 2s 78ms/step - loss: 0.7388 - accuracy: 0.5137 - val_loss: 0.8462 - val_accuracy: 0.3295 - lr: 0.0010
Epoch 6/50
32/32 [==============================] - 3s 79ms/step - loss: 0.7258 - accuracy: 0.5130 - val_loss: 0.8065 - val_accuracy: 0.3523 - lr: 0.0010
Epoch 7/50
32/32 [==============================] - 2s 78ms/step - loss: 0.

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [29]:
# Define the Model

model = Sequential([
    Conv2D(32, (1,1), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (1,1), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (1,1), activation='relu'),
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')

])

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard

import os

def scheduler(epoch):
  if epoch < 10:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (10 - epoch))


In [30]:
# Compile Model
lrs = LearningRateScheduler(scheduler)
logdir = os.path.join("/logs","adadelta" )

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
# Fit Model
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[lrs]
)

Epoch 1/100
32/32 [==============================] - 2s 57ms/step - loss: 0.6925 - accuracy: 0.4870 - val_loss: 0.6903 - val_accuracy: 0.6420 - lr: 0.0010
Epoch 2/100
32/32 [==============================] - 2s 58ms/step - loss: 0.6925 - accuracy: 0.4770 - val_loss: 0.6901 - val_accuracy: 0.6648 - lr: 0.0010
Epoch 3/100
32/32 [==============================] - 2s 57ms/step - loss: 0.6923 - accuracy: 0.4970 - val_loss: 0.6902 - val_accuracy: 0.6648 - lr: 0.0010
Epoch 4/100
32/32 [==============================] - 2s 57ms/step - loss: 0.6924 - accuracy: 0.4870 - val_loss: 0.6901 - val_accuracy: 0.6761 - lr: 0.0010
Epoch 5/100
32/32 [==============================] - 2s 58ms/step - loss: 0.6924 - accuracy: 0.4790 - val_loss: 0.6903 - val_accuracy: 0.6591 - lr: 0.0010
Epoch 6/100
32/32 [==============================] - 2s 60ms/step - loss: 0.6924 - accuracy: 0.4790 - val_loss: 0.6904 - val_accuracy: 0.6705 - lr: 0.0010
Epoch 7/100
32/32 [==============================] - 2s 58ms/step - lo

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [13]:
# Let's start with the baseline put together in the training article

train_image_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    rescale=1./255,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'
    ) # Generator for our training data

validation_image_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    rescale=1./255,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'
) # Generator for our validation data



In [14]:
# Create our train and validation generators

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 533 images belonging to 2 classes.
Found 195 images belonging to 2 classes.


In [19]:
# Define our baseline model
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, Sequential # This is the functional API

model = Sequential([
    Conv2D(32, (1, 1), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (1, 1), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (1, 1), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard

import os

def scheduler(epoch):
  if epoch < 10:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (10 - epoch))

In [20]:
# Compile our model

lrs = LearningRateScheduler(scheduler)
logdir = os.path.join("/logs","adadelta" )

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[lrs]
)
model.save_weights('first_try.h5')

Epoch 1/100
32/32 [==============================] - 9s 268ms/step - loss: 0.6495 - accuracy: 0.6786 - val_loss: 0.6181 - val_accuracy: 0.7784 - lr: 0.0010
Epoch 2/100
32/32 [==============================] - 9s 267ms/step - loss: 0.5607 - accuracy: 0.7844 - val_loss: 0.5033 - val_accuracy: 0.8693 - lr: 0.0010
Epoch 3/100
32/32 [==============================] - 9s 266ms/step - loss: 0.4889 - accuracy: 0.8024 - val_loss: 0.4244 - val_accuracy: 0.8580 - lr: 0.0010
Epoch 4/100
32/32 [==============================] - 9s 273ms/step - loss: 0.4352 - accuracy: 0.8423 - val_loss: 0.3838 - val_accuracy: 0.8352 - lr: 0.0010
Epoch 5/100
32/32 [==============================] - 8s 265ms/step - loss: 0.4039 - accuracy: 0.8463 - val_loss: 0.3818 - val_accuracy: 0.8409 - lr: 0.0010
Epoch 6/100
32/32 [==============================] - 9s 266ms/step - loss: 0.4087 - accuracy: 0.8363 - val_loss: 0.4050 - val_accuracy: 0.8352 - lr: 0.0010
Epoch 7/100
32/32 [==============================] - 9s 269ms/st

Perfect! Our model was able to reach up to a 93% accuracy with the minimal amount of data that it was provided!

In [46]:
!wget https://scx2.b-cdn.net/gfx/news/2018/europeslostf.jpg

--2020-06-18 03:36:39--  https://scx2.b-cdn.net/gfx/news/2018/europeslostf.jpg
Resolving scx2.b-cdn.net (scx2.b-cdn.net)... 84.17.46.50, 2a02:6ea0:c020::2
Connecting to scx2.b-cdn.net (scx2.b-cdn.net)|84.17.46.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453310 (443K) [image/jpeg]
Saving to: ‘europeslostf.jpg.1’

europeslostf.jpg.1  100%[===================>] 442.69K  --.-KB/s    in 0.02s   

2020-06-18 03:36:39 (22.3 MB/s) - ‘europeslostf.jpg.1’ saved [453310/453310]



In [52]:
# Load in our test image
from keras.preprocessing.image import load_img
img = load_img('europeslostf.jpg.1', target_size=(IMG_HEIGHT, IMG_WIDTH))
print(type(img))
print(img.format)
print(img.mode)
print(img.size)

<class 'PIL.Image.Image'>
None
RGB
(224, 224)


In [54]:
# Convert our test image to an array
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

test_img_array = img_to_array(img)
print(test_img_array.dtype)
print(test_img_array.shape)

float32
(224, 224, 3)


In [62]:
test_img_array = test_img_array.reshape([1, IMG_HEIGHT, IMG_WIDTH, 3])

In [63]:
model.predict(test_img_array)

array([[7.197398e-17]], dtype=float32)

In [64]:
# Try again using an image of a mountain.
!wget https://miro.medium.com/max/4320/1*8GDFUEsd0z8SidRfg9IljA.png

--2020-06-18 04:02:17--  https://miro.medium.com/max/4320/1*8GDFUEsd0z8SidRfg9IljA.png
Resolving miro.medium.com (miro.medium.com)... 104.16.121.145, 104.16.117.145, 104.16.118.145, ...
Connecting to miro.medium.com (miro.medium.com)|104.16.121.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4661901 (4.4M) [image/png]
Saving to: ‘1*8GDFUEsd0z8SidRfg9IljA.png’

1*8GDFUEsd0z8SidRfg 100%[===================>]   4.45M  27.7MB/s    in 0.2s    

2020-06-18 04:02:18 (27.7 MB/s) - ‘1*8GDFUEsd0z8SidRfg9IljA.png’ saved [4661901/4661901]



In [65]:
# Load in our mountain test image
from keras.preprocessing.image import load_img
mount_img = load_img('1*8GDFUEsd0z8SidRfg9IljA.png', target_size=(IMG_HEIGHT, IMG_WIDTH))
print(type(mount_img))
print(mount_img.format)
print(mount_img.mode)
print(mount_img.size)

<class 'PIL.Image.Image'>
None
RGB
(224, 224)


In [66]:
# Convert our test image to an array
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

mount_img_array = img_to_array(mount_img)
print(mount_img_array.dtype)
print(mount_img_array.shape)

mount_img_array = mount_img_array.reshape([1, IMG_HEIGHT, IMG_WIDTH, 3])

float32
(224, 224, 3)


In [67]:
# Make our prediction!
model.predict(mount_img_array)

array([[1.]], dtype=float32)

# Results

Our output for the test img (a forest) is very close to zero. In our setup we are depicting forests as an output of 0, so our model correctly identified a completely new picture! Our second test fed the model a completely new picture of a mountain. Our model output 1. In our setup we are depicting mountains as an ouput of 1, so our model correctly identified a second new picture!

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language